In [58]:
import re

with open("inputs/Day_14.txt") as f:
    puzzle_data = f.read()
    
    
def part_1_solution(raw_data):
    memory = dict()
    mask = None
    
    for raw_line in raw_data.splitlines():
        if raw_line.startswith('mask'):
            mask = parse_mask(raw_line)
            continue
            
        memory_idx, init_value = parse_memory_instruction(raw_line)
        memory[memory_idx] = apply_mask(init_value, mask)
        
    return sum(memory.values())

def parse_mask(raw_mask):
    mask = dict()
    _, mask_value = raw_mask.split('=')
    mask_value = mask_value.strip()

    for idx, value in enumerate(mask_value):
        if value != 'X':
            mask[idx] = value
    return mask

def parse_memory_instruction(raw_instruction):
    match = re.match(r'mem\[(?P<idx>\d*)\] = (?P<value>\d*)', raw_instruction)
    return int(match['idx']), int(match['value'])

def apply_mask(value, mask):
    value_bin = list(f"{value:036b}")
    for idx, bit in mask.items():
        value_bin[idx] = bit
    
    return int("".join(value_bin), 2)

In [60]:
from helpers import test_single_case

test_input = """\
mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0\
"""
test_single_case(part_1_solution, 165, test_input)

PASSED (in 0.06 [ms])


In [61]:
%%time
print(f"Part 1 solution: {part_1_solution(puzzle_data)}")

Part 1 solution: 12512013221615
CPU times: user 3.48 ms, sys: 0 ns, total: 3.48 ms
Wall time: 3.45 ms


In [18]:
import re

with open("inputs/Day_14.txt") as f:
    puzzle_data = f.read()
    
    
def part_2_solution(raw_data):
    memory = dict()
    mask = None
    
    for raw_line in raw_data.splitlines():
        if raw_line.startswith('mask'):
            mask = parse_mask(raw_line)
            continue
            
        memory_idx, value = parse_memory_instruction(raw_line)
        
        possible_idx = apply_mask(memory_idx, mask)
        
        for memory_idx in possible_idx:
            memory[memory_idx] = value
        
    return sum(memory.values())

def parse_mask(raw_mask):
    mask = dict()
    _, mask_value = raw_mask.split('=')
    mask_value = mask_value.strip()
    
    return mask_value

def parse_memory_instruction(raw_instruction):
    match = re.match(r'mem\[(?P<idx>\d*)\] = (?P<value>\d*)', raw_instruction)
    return int(match['idx']), int(match['value'])

def apply_mask(value, mask):
    def add_char_to_all_numbers(char_to_add, numbers):
        if not numbers:
            return [[char_to_add]]

        new_numbers = list()
        for number in numbers:
            new_number = number[:]
            new_number.append(char_to_add)
            new_numbers.append(new_number)

        return new_numbers
    
    value_bin = list(f"{value:036b}")
    possible_values = list()
    
    for idx, char in enumerate(mask):
        if char == 'X':
            numbers_with_added_0 = add_char_to_all_numbers('0', possible_values)
            numbers_with_added_1 = add_char_to_all_numbers('1', possible_values)
            possible_values = numbers_with_added_0
            possible_values.extend(numbers_with_added_1)
        elif char == '1':
            possible_values = add_char_to_all_numbers('1', possible_values)
        elif char == '0':
            possible_values = add_char_to_all_numbers(value_bin[idx], possible_values)
        else:
            raise Exception(f"Unknown char: {char}")
    
    return [int("".join(possible_value), 2) for possible_value in possible_values]

In [19]:
from helpers import test_single_case

test_input = """\
mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1\
"""
test_single_case(part_2_solution, 208, test_input)

PASSED (in 0.13 [ms])


In [20]:
%%time
print(f"Part 2 solution: {part_2_solution(puzzle_data)}")

Part 2 solution: 3905642473893
CPU times: user 408 ms, sys: 9.2 ms, total: 417 ms
Wall time: 415 ms
